<a href="https://colab.research.google.com/github/KoNGoL/tfm_breast_cancer_detection/blob/main/Retinanet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!mkdir -p /content/Drive/
from google.colab import drive
drive.mount('/content/Drive') 
!mkdir -p /content/Drive/MyDrive/RetinaNet/
%cd /content/Drive/MyDrive/RetinaNet/

In [ ]:
!git clone https://github.com/fizyr/keras-retinanet
%cd keras-retinanet/
!pip install numpy --user
!pip install . --user
!python setup.py build_ext --inplace
!wget https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5
!mv  /content/Drive/MyDrive/RetinaNet/keras-retinanet/resnet50_coco_best_v2.1.0.h5  /content/Drive/MyDrive/RetinaNet//keras-retinanet/snapshots/


In [ ]:
%cd examples 
!python resnet50_retinanet.py

/content/keras-retinanet/examples
Traceback (most recent call last):
  File "resnet50_retinanet.py", line 31, in <module>
    setup_gpu(gpu)
  File "../keras_retinanet/utils/gpu.py", line 22, in setup_gpu
    visible_gpu_indices = [int(id) for id in gpu_id.split(',')]
AttributeError: 'int' object has no attribute 'split'


In [6]:
%cd /content
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="yGM3vBhGiZQUJrW3szSn")
project = rf.workspace("tfm-pq95g").project("tfm-breast_640_separated")
dataset = project.version(1).download("voc")

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached roboflow-0.2.18-py3-none-any.whl (41 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)
  Using cached certifi-2021.5.30-py2.py3-none-any.whl (145 kB)
  Using cached wget-3.2-py3-none-any.whl
  Using cached requests_toolbelt-0.10.1-py2.py3-none-any.whl (54 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached urllib3-1.26.6-py2.py3-none-any.whl (138 kB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: certifi
    Found existing installation: certifi 2022.9.24
    Uninstalling certifi-2022.9.24:
      Successfully uninstalled

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to TFM-breast_640_separated-1 in voc:: 100%|██████████| 11645/11645 [00:02<00:00, 4921.48it/s]


In [7]:
!mkdir /content/Drive/MyDrive/RetinaNet/dataset
!mkdir /content/Drive/MyDrive/RetinaNet/dataset/{Annotations,ImageSets,JPEGImages}
!mkdir /content/Drive/MyDrive/RetinaNet/dataset/ImageSets/Main
!cp /content/TFM-breast_640_separated-1/test/*jpg /content/Drive/MyDrive/RetinaNet/dataset/JPEGImages/
!cp /content/TFM-breast_640_separated-1/train/*jpg /content/Drive/MyDrive/RetinaNet/dataset/JPEGImages/
!cp /content/TFM-breast_640_separated-1/valid/*jpg /content/Drive/MyDrive/RetinaNet/dataset/JPEGImages/
!cp /content/TFM-breast_640_separated-1/train/*xml /content/Drive/MyDrive/RetinaNet/dataset/Annotations/
!cp /content/TFM-breast_640_separated-1/test/*xml /content/Drive/MyDrive/RetinaNet/dataset/Annotations/
!cp /content/TFM-breast_640_separated-1/valid/*xml /content/Drive/MyDrive/RetinaNet/dataset/Annotations/

In [9]:
from os import listdir
from os.path import isfile, join

# dataset_directory = "/content/dataset/Annotations"

with open("/content/Drive/MyDrive/RetinaNet/dataset/ImageSets/Main/test.txt", "w") as test:
    for file in listdir('/content/TFM-breast_640_separated-1/test/'):
        if ".xml" in file:
          continue
        name = file.split("\\")[-1]
        name = name.replace('.jpg', '')
        test.write(name + "\n")
        # print("test:    " + name)
        
with open("/content/Drive/MyDrive/RetinaNet/dataset/ImageSets/Main/train.txt", "w") as train:
    for file in listdir('/content/TFM-breast_640_separated-1/train/'):
        if ".xml" in file:
          continue
        name = file.split("\\")[-1]
        name = name.replace('.jpg', '')
        train.write(name + "\n")   
        # print("train:    " + name)   

with open("/content/Drive/MyDrive/RetinaNet/dataset/ImageSets/Main/val.txt", "w") as val:
    for file in listdir('/content/TFM-breast_640_separated-1/valid/'):
        if ".xml" in file:
          continue
        name = file.split("\\")[-1]
        name = name.replace('.jpg', '')
        val.write(name + "\n")   
        # print("train:    " + name)

test.close()
train.close()
val.close()

In [13]:
!python /content/Drive/MyDrive/RetinaNet/keras-retinanet/keras_retinanet/bin/train.py --imagenet-weights --gpu 0 --epoch 100 --batch-size 8 --steps 614 --snapshot-path /content/Drive/MyDrive/RetinaNet/retinanet_models/ --tensorboard-dir /content/Drive/MyDrive/RetinaNet/retinanet_log/ pascal /content/Drive/MyDrive/RetinaNet/dataset/

1 Physical GPUs, 1 Logical GPUs
Creating model, this may take a second...
/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
Model: "retinanet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1 (Conv2D)                 (None, None, None,   9408        ['input_1[0][0]']                
                                64)                                                               
            